In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
price = pd.read_excel('fama_french_data.xlsx',sheet_name='수정주가',index_col=0)
amounts = pd.read_excel('fama_french_data.xlsx',sheet_name='상장주식수',index_col=0)
pbr = pd.read_excel('fama_french_data.xlsx',sheet_name='pbr',index_col=0)
returns = pd.read_excel('fama_french_data.xlsx',sheet_name='수익률',index_col=0)
rf = pd.read_excel('fama_french_data.xlsx',sheet_name='금리',index_col=0)
index_returns = pd.read_excel('fama_french_data.xlsx',sheet_name='지수수익률',index_col=0)

In [3]:
data = price * amounts      # 시가총액 계산입니다.
inverse_pbr = 1/pbr     # 지표를 사용하기 위하여 pbr의 역수를 취해줍니다.
returns = returns.replace(np.NaN,0)   # 수익률에서 NaN 값은 모두 0으로 처리합니다.

In [4]:
weight = data.copy()

In [5]:
def weighted_returns(portfolio):        # 함수를 하나 정의하겠습니다.

    weights = weight[portfolio]     # 비중을 계산하기 위해 위에서 data를 복사한 weight에 계산하고자하는 포트폴리오를 넣어줍니다.
    weights['sum'] = weights.sum(axis = 1)      # 이후 열의 합계를 구해 데이터프레임의 sum이라는 칼럼에 넣어줍니다.

    for columns in weights.columns:     # 이후 반복문에 넣어 비중을 계산합니다.
        weights[columns] = weights[columns] / weights['sum']

    weights.drop(columns='sum',inplace=True)        # 이제 sum이라는 칼럼은 필요없으니 버려줍니다.
    weighted_return = weights * returns[portfolio]      # 이제 수익률의 가중평균(비중 * 수익률)을 하기 위해 비중과 포트폴리오의 수익률을 곱해줍니다.
    return weighted_return

In [6]:
SMB_ls = []
HML_ls = []
sh_return = []
sm_return = []
sv_return = []
hh_return = []
hm_return = []
hv_return = []


for i in range(len(data.index)):
    capital = data.iloc[i,:].dropna().sort_values(ascending=True)
    percentile_value = np.percentile(capital, 50)

    # 소형주, 대형주(S,H) 구분
    big = capital[capital >= percentile_value].index    # 중앙값보다 큰 경우 대형주로 분류하겠습니다.
    small = capital[capital < percentile_value].index   # 중앙값보다 작은 경우 소형주로 분류하겠습니다.

    value_big = inverse_pbr.loc[:,big].iloc[i,:].sort_values(ascending = True)     # value factor을 측정하기 위함(pbr의 역수에서 오름차순으로 나열하면, 맨 위에는 고 pbr 종목이 나옵니다.)
    big_len = len(value_big)

    big_high = value_big[:int(big_len * 0.3)].index  # 첫 번째 30% 부분 (고 pbr)
    big_netural = value_big[int(big_len * 0.3):int(big_len * 0.7)].index  # 중간 40% 부분
    big_value = value_big[int(big_len * 0.7):].index  # 나머지 30% 부분 (저 pbr)

    value_small = inverse_pbr.loc[:,small].iloc[i,:].sort_values(ascending = True)     # value factor을 측정하기 위함(pbr의 역수에서 오름차순으로 나열하면, 맨 위에는 고 pbr 종목이 나옵니다.)
    small_len = len(value_small)

    small_high = value_small[:int(big_len * 0.3)].index  # 첫 번째 30% 부분 (고 pbr)
    small_netural = value_small[int(big_len * 0.3):int(big_len * 0.7)].index  # 중간 40% 부분
    small_value = value_small[int(big_len * 0.7):].index  # 나머지 30% 부분 (저 pbr)


    # 각 포트폴리오의 수익률 평균 계산
    sh_returns = weighted_returns(small_high)
    sh_returns = sh_returns.iloc[i,:].sum()

    sm_returns = weighted_returns(small_netural)
    sm_returns = sm_returns.iloc[i,:].sum()

    sv_returns = weighted_returns(small_value)
    sv_returns = sv_returns.iloc[i,:].sum()

    bh_returns = weighted_returns(big_high)
    bh_returns = bh_returns.iloc[i,:].sum()

    bm_returns = weighted_returns(big_netural)
    bm_returns = bm_returns.iloc[i,:].sum()

    bv_returns = weighted_returns(big_value)
    bv_returns = bv_returns.iloc[i,:].sum()

    # SML, HML 계산
    SMB = (sh_returns + sm_returns + sv_returns) / 3 - (bh_returns + bm_returns + bv_returns) / 3
    HML = (sh_returns + bh_returns) / 2 - (sv_returns + bv_returns) / 2

    # 리스트에 담기
    SMB_ls.append(SMB)
    HML_ls.append(HML)
    sh_return.append(sh_returns)
    sm_return.append(sm_returns)
    sv_return.append(sv_returns)
    hh_return.append(bh_returns)
    hm_return.append(bm_returns)
    hv_return.append(bv_returns)

C:\Users\82105\AppData\Local\Temp\ipykernel_13304\2197174297.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weights['sum'] = weights.sum(axis = 1)      # 이후 열의 합계를 구해 데이터프레임의 sum이라는 칼럼에 넣어줍니다.
C:\Users\82105\AppData\Local\Temp\ipykernel_13304\2197174297.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weights[columns] = weights[columns] / weights['sum']
C:\Users\82105\AppData\Local\Temp\ipykernel_13304\2197174297.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [7]:
rf['SMB'] = SMB_ls
rf['HML'] = HML_ls
rf['Rm-Rf'] = index_returns['Total'] - rf['Interest Rate']      # 주식 시장의 프리미엄(시장수익률 - 무위험수익률)을 구해줍니다.
rf['SH'] = sh_return       # 데이터 프레임에 각각의 값을 넣어줍니다.
rf['SM'] = sm_return
rf['SV'] = sv_return
rf['BH'] = hh_return
rf['BM'] = hm_return
rf['HV'] = hv_return
final_data = rf.drop(columns='Interest Rate')   # 무위험 수익률은 날려주고 final_data를 만듭니다.

In [8]:
final_data      # 이렇게 생겼음을 알 수 있습니다. #iloc 나 loc 써서 하기

,SMB,HML,Rm-Rf,SH,SM,SV,BH,BM,HV
Symbol,,,,,,,,,
2003-01-31,0.208219,1.964137,-9.180437,-0.638434,-4.819370,-4.609543,-6.347713,1.960588,-6.304879
2003-02-28,-0.292197,1.608034,-7.871444,-0.938914,-2.165383,1.129265,5.000098,-5.814390,-0.284149
2003-03-31,5.278258,3.761841,-10.239913,-13.659557,-12.214589,-11.672151,-6.124284,-31.621414,-15.635372
2003-04-30,1.504527,-10.375890,2.458849,12.438050,8.947352,9.689976,0.800035,1.461873,24.299888
2003-05-31,0.267543,-2.111929,0.903344,12.126667,7.734256,4.852196,1.995436,8.421288,13.493766
...,...,...,...,...,...,...,...,...,...
2023-06-30,0.059973,3.378403,-4.180323,2.751168,0.194431,-1.058068,1.930188,0.794809,-1.017383
2023-07-31,-14.688102,11.404770,-1.314632,-6.095657,-7.991020,-7.523093,19.876810,4.083021,-1.505294
2023-08-31,2.618874,0.988099,-5.873429,2.789790,1.166239,0.499113,-0.581139,-2.553681,-0.266661


In [9]:
X = final_data[['SMB','HML','Rm-Rf']]       # 모델적합
y = final_data['BH'] - rf['Interest Rate']

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.776
Model:                            OLS   Adj. R-squared (uncentered):              0.773
Method:                 Least Squares   F-statistic:                              285.5
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    5.82e-80
Time:                        15:50:59   Log-Likelihood:                         -745.60
No. Observations:                 250   AIC:                                      1497.
Df Residuals:                     247   BIC:                                      1508.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB           -0.7394      0.052    -14.129      0.000      -0.842      -0.636
HML            0.7735      0.040     19.507      0.000       0.695       0.852
Rm-Rf          1.1875      0.058     20.381      0.000       1.073       1.302
==============================================================================
Omnibus:                       14.042   Durbin-Watson:                   2.048
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               36.449
Skew:                          -0.044   Prob(JB):                     1.22e-08
Kurtosis:                       4.869   Cond. No.                         1.56
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
X = final_data[['SMB','HML','Rm-Rf']]
y = final_data['SH'] - rf['Interest Rate']

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.551
Model:                            OLS   Adj. R-squared (uncentered):              0.546
Method:                 Least Squares   F-statistic:                              101.1
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    9.89e-43
Time:                        15:50:59   Log-Likelihood:                         -772.39
No. Observations:                 250   AIC:                                      1551.
Df Residuals:                     247   BIC:                                      1561.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.3070      0.058      5.271      0.000       0.192       0.422
HML            0.1881      0.044      4.261      0.000       0.101       0.275
Rm-Rf          1.0877      0.065     16.771      0.000       0.960       1.215
==============================================================================
Omnibus:                       50.643   Durbin-Watson:                   1.364
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              145.539
Skew:                           0.871   Prob(JB):                     2.49e-32
Kurtosis:                       6.307   Cond. No.                         1.56
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
X = final_data[['SMB','HML','Rm-Rf']]
y = final_data['BM'] - rf['Interest Rate']

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.495
Model:                            OLS   Adj. R-squared (uncentered):              0.489
Method:                 Least Squares   F-statistic:                              80.77
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    1.92e-36
Time:                        15:50:59   Log-Likelihood:                         -825.31
No. Observations:                 250   AIC:                                      1657.
Df Residuals:                     247   BIC:                                      1667.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB           -0.3896      0.072     -5.413      0.000      -0.531      -0.248
HML            0.0797      0.055      1.461      0.145      -0.028       0.187
Rm-Rf          1.1356      0.080     14.169      0.000       0.978       1.293
==============================================================================
Omnibus:                      263.633   Durbin-Watson:                   2.287
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22375.807
Skew:                           3.895   Prob(JB):                         0.00
Kurtosis:                      48.688   Cond. No.                         1.56
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
X = final_data[['SMB','HML','Rm-Rf']]
y = final_data['SM'] - rf['Interest Rate']

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.738
Model:                            OLS   Adj. R-squared (uncentered):              0.735
Method:                 Least Squares   F-statistic:                              231.8
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    1.61e-71
Time:                        15:50:59   Log-Likelihood:                         -694.17
No. Observations:                 250   AIC:                                      1394.
Df Residuals:                     247   BIC:                                      1405.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.3799      0.043      8.918      0.000       0.296       0.464
HML           -0.0277      0.032     -0.857      0.392      -0.091       0.036
Rm-Rf          1.1769      0.047     24.813      0.000       1.084       1.270
==============================================================================
Omnibus:                       10.621   Durbin-Watson:                   1.742
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               19.412
Skew:                           0.176   Prob(JB):                     6.09e-05
Kurtosis:                       4.319   Cond. No.                         1.56
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
X = final_data[['SMB','HML','Rm-Rf']]
y = final_data['HV'] - rf['Interest Rate']

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.762
Model:                            OLS   Adj. R-squared (uncentered):              0.759
Method:                 Least Squares   F-statistic:                              264.0
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    9.86e-77
Time:                        15:51:00   Log-Likelihood:                         -817.06
No. Observations:                 250   AIC:                                      1640.
Df Residuals:                     247   BIC:                                      1651.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB           -0.8082      0.070    -11.605      0.000      -0.945      -0.671
HML           -0.8656      0.053    -16.402      0.000      -0.970      -0.762
Rm-Rf          1.1084      0.078     14.293      0.000       0.956       1.261
==============================================================================
Omnibus:                       44.547   Durbin-Watson:                   1.691
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              121.179
Skew:                           0.781   Prob(JB):                     4.86e-27
Kurtosis:                       6.032   Cond. No.                         1.56
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
X = final_data[['SMB','HML','Rm-Rf']]
y = final_data['SV'] - rf['Interest Rate']

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.775
Model:                            OLS   Adj. R-squared (uncentered):              0.772
Method:                 Least Squares   F-statistic:                              283.8
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    1.01e-79
Time:                        15:51:00   Log-Likelihood:                         -677.88
No. Observations:                 250   AIC:                                      1362.
Df Residuals:                     247   BIC:                                      1372.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.3759      0.040      9.418      0.000       0.297       0.455
HML           -0.1728      0.030     -5.714      0.000      -0.232      -0.113
Rm-Rf          1.1668      0.044     26.257      0.000       1.079       1.254
==============================================================================
Omnibus:                       15.782   Durbin-Watson:                   1.869
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               44.541
Skew:                           0.071   Prob(JB):                     2.13e-10
Kurtosis:                       5.063   Cond. No.                         1.56
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
# 소형주 그룹에서 대형주 그룹으로 갈수록 SMB 기울기 s는 낮아진다. 저-BE/ME(V) 그룹에서 고-BE/ME(H) 그룹으로 갈수록 HML 기울기 h는 높아진다.
# 위의 문장이 3팩터의 회귀계수(coefficient)의 해석입니다.

# 해석은 자율적으로 해볼까요?


In [16]:
len(X)

250

In [17]:
len(y)

250

In [18]:
# 2003.9 ~ 2008.8

In [19]:
final_data1 = final_data.iloc[8:68]
final_data1

,SMB,HML,Rm-Rf,SH,SM,SV,BH,BM,HV
Symbol,,,,,,,,,
2003-09-30,5.140789,-1.338290,-10.975525,-6.202290,-6.441294,-6.767610,-13.767146,-10.541168,-10.525246
2003-10-31,-2.394591,8.564115,7.151154,-0.272538,-2.143888,-5.054875,4.170044,3.718277,-8.175849
2003-11-30,8.771382,10.969387,-1.332741,0.802090,-0.255349,-0.579950,-2.321210,-1.148200,-22.877945
2003-12-31,10.008580,4.079792,-2.103200,3.440157,-1.168687,0.286745,-13.282766,4.104177,-18.288938
2004-01-31,3.506685,5.384524,0.482118,2.157134,-4.203794,-6.478689,-10.378655,3.845133,-12.511881
2004-02-29,-12.801376,-11.430002,-1.591922,2.790143,-0.764375,-1.887742,15.253253,-19.502239,42.791141
2004-03-31,1.404508,3.177920,-4.051022,2.172055,-0.480371,-4.294286,-2.432281,-2.062062,-2.321781
2004-04-30,8.441520,15.582010,-4.866273,8.948361,2.563592,-0.489369,4.745034,-2.065753,-16.981256
2004-05-31,-2.919376,1.491895,-10.027702,-9.627622,-9.075841,-8.966120,-5.105959,-5.054248,-8.751249


In [20]:
X = final_data1[['SMB','HML','Rm-Rf']]       # 모델적합
y = final_data1['BH'] - rf['Interest Rate'].iloc[8:68]

In [21]:
len(X)

60

In [22]:
len(y)

60

In [23]:
X = final_data1[['SMB','HML','Rm-Rf']]       # 모델적합
y = final_data1['BH'] - rf['Interest Rate'].iloc[8:68]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.774
Model:                            OLS   Adj. R-squared (uncentered):              0.763
Method:                 Least Squares   F-statistic:                              65.23
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    2.00e-18
Time:                        15:51:00   Log-Likelihood:                         -199.87
No. Observations:                  60   AIC:                                      405.7
Df Residuals:                      57   BIC:                                      412.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB           -0.8986      0.097     -9.243      0.000      -1.093      -0.704
HML            0.7874      0.085      9.301      0.000       0.618       0.957
Rm-Rf          1.1485      0.145      7.938      0.000       0.859       1.438
==============================================================================
Omnibus:                        3.093   Durbin-Watson:                   2.239
Prob(Omnibus):                  0.213   Jarque-Bera (JB):                2.301
Skew:                          -0.282   Prob(JB):                        0.317
Kurtosis:                       3.776   Cond. No.                         1.89
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
X = final_data1[['SMB','HML','Rm-Rf']]
y = final_data1['SH'] - rf['Interest Rate'].iloc[8:68]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.439
Model:                            OLS   Adj. R-squared (uncentered):              0.410
Method:                 Least Squares   F-statistic:                              14.89
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    2.85e-07
Time:                        15:51:00   Log-Likelihood:                         -197.21
No. Observations:                  60   AIC:                                      400.4
Df Residuals:                      57   BIC:                                      406.7
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.1262      0.093      1.357      0.180      -0.060       0.312
HML            0.0716      0.081      0.885      0.380      -0.091       0.234
Rm-Rf          0.9106      0.138      6.579      0.000       0.633       1.188
==============================================================================
Omnibus:                       29.253   Durbin-Watson:                   1.286
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               76.585
Skew:                           1.404   Prob(JB):                     2.34e-17
Kurtosis:                       7.769   Cond. No.                         1.89
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
X = final_data1[['SMB','HML','Rm-Rf']]
y = final_data1['BM'] - rf['Interest Rate'].iloc[8:68]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.326
Model:                            OLS   Adj. R-squared (uncentered):              0.291
Method:                 Least Squares   F-statistic:                              9.200
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    4.67e-05
Time:                        15:51:00   Log-Likelihood:                         -230.70
No. Observations:                  60   AIC:                                      467.4
Df Residuals:                      57   BIC:                                      473.7
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB           -0.5819      0.163     -3.581      0.001      -0.907      -0.256
HML            0.1895      0.142      1.339      0.186      -0.094       0.473
Rm-Rf          0.8908      0.242      3.683      0.001       0.407       1.375
==============================================================================
Omnibus:                       52.986   Durbin-Watson:                   2.359
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              403.271
Skew:                           2.199   Prob(JB):                     2.70e-88
Kurtosis:                      14.915   Cond. No.                         1.89
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
X = final_data1[['SMB','HML','Rm-Rf']]
y = final_data1['SM'] - rf['Interest Rate'].iloc[8:68]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.658
Model:                            OLS   Adj. R-squared (uncentered):              0.640
Method:                 Least Squares   F-statistic:                              36.50
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    2.70e-13
Time:                        15:51:00   Log-Likelihood:                         -178.26
No. Observations:                  60   AIC:                                      362.5
Df Residuals:                      57   BIC:                                      368.8
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.1970      0.068      2.906      0.005       0.061       0.333
HML           -0.0443      0.059     -0.751      0.456      -0.163       0.074
Rm-Rf          1.0188      0.101     10.095      0.000       0.817       1.221
==============================================================================
Omnibus:                        6.428   Durbin-Watson:                   1.487
Prob(Omnibus):                  0.040   Jarque-Bera (JB):                6.779
Skew:                           0.446   Prob(JB):                       0.0337
Kurtosis:                       4.385   Cond. No.                         1.89
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
X = final_data1[['SMB','HML','Rm-Rf']]
y = final_data1['HV'] - rf['Interest Rate'].iloc[8:68]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.829
Model:                            OLS   Adj. R-squared (uncentered):              0.820
Method:                 Least Squares   F-statistic:                              92.16
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    7.60e-22
Time:                        15:51:00   Log-Likelihood:                         -212.10
No. Observations:                  60   AIC:                                      430.2
Df Residuals:                      57   BIC:                                      436.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB           -0.9843      0.119     -8.258      0.000      -1.223      -0.746
HML           -1.0452      0.104    -10.070      0.000      -1.253      -0.837
Rm-Rf          0.9746      0.177      5.494      0.000       0.619       1.330
==============================================================================
Omnibus:                        4.143   Durbin-Watson:                   1.923
Prob(Omnibus):                  0.126   Jarque-Bera (JB):                3.713
Skew:                           0.298   Prob(JB):                        0.156
Kurtosis:                       4.063   Cond. No.                         1.89
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
X = final_data1[['SMB','HML','Rm-Rf']]
y = final_data1['SV'] - rf['Interest Rate'].iloc[8:68]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.680
Model:                            OLS   Adj. R-squared (uncentered):              0.663
Method:                 Least Squares   F-statistic:                              40.37
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    4.01e-14
Time:                        15:51:00   Log-Likelihood:                         -179.72
No. Observations:                  60   AIC:                                      365.4
Df Residuals:                      57   BIC:                                      371.7
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.2120      0.069      3.051      0.003       0.073       0.351
HML           -0.0958      0.061     -1.583      0.119      -0.217       0.025
Rm-Rf          1.0845      0.103     10.488      0.000       0.877       1.292
==============================================================================
Omnibus:                        5.188   Durbin-Watson:                   1.691
Prob(Omnibus):                  0.075   Jarque-Bera (JB):                6.437
Skew:                           0.187   Prob(JB):                       0.0400
Kurtosis:                       4.561   Cond. No.                         1.89
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
#2008.9 ~ 2013.8

In [30]:
final_data2 = final_data.iloc[68:128]
final_data2

,SMB,HML,Rm-Rf,SH,SM,SV,BH,BM,HV
Symbol,,,,,,,,,
2008-09-30,5.406030,17.499446,-6.881065,-5.052272,-7.965091,-9.325268,-2.775341,-2.284146,-33.501236
2008-10-31,5.678978,16.032083,-25.592798,-27.951145,-31.530790,-37.087203,-24.764847,-41.148270,-47.692955
2008-11-30,2.522821,7.503337,3.158538,21.722455,14.641730,6.269601,12.852682,8.913781,13.298861
2008-12-31,-1.891833,-0.205730,1.166843,9.952122,8.019142,5.236515,7.730778,8.294658,12.857844
2009-01-31,4.891800,16.219065,0.584538,8.432698,8.184880,1.371402,10.347524,7.995367,-15.029311
2009-02-28,7.062925,13.702536,-4.540705,7.592665,2.850851,-0.131169,3.999431,0.805948,-15.681807
2009-03-31,3.359902,9.095936,10.282913,20.466544,13.078485,7.282301,12.859460,10.036335,7.851830
2009-04-30,-1.119809,-0.585724,11.018180,34.314392,24.977154,20.146545,21.664959,24.128307,37.004253
2009-05-31,-8.840244,2.657626,1.868664,10.168580,6.817583,5.239779,9.490779,30.151567,9.104328


In [31]:
X = final_data2[['SMB','HML','Rm-Rf']]
y = final_data2['BH'] - rf['Interest Rate'].iloc[68:128]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.819
Model:                            OLS   Adj. R-squared (uncentered):              0.809
Method:                 Least Squares   F-statistic:                              85.73
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    4.13e-21
Time:                        15:51:00   Log-Likelihood:                         -175.67
No. Observations:                  60   AIC:                                      357.3
Df Residuals:                      57   BIC:                                      363.6
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB           -0.8207      0.132     -6.202      0.000      -1.086      -0.556
HML            0.7248      0.070     10.355      0.000       0.585       0.865
Rm-Rf          1.1492      0.111     10.321      0.000       0.926       1.372
==============================================================================
Omnibus:                        2.357   Durbin-Watson:                   2.137
Prob(Omnibus):                  0.308   Jarque-Bera (JB):                1.454
Skew:                           0.061   Prob(JB):                        0.483
Kurtosis:                       2.247   Cond. No.                         2.23
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
X = final_data2[['SMB','HML','Rm-Rf']]
y = final_data2['SH'] - rf['Interest Rate'].iloc[68:128]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.609
Model:                            OLS   Adj. R-squared (uncentered):              0.588
Method:                 Least Squares   F-statistic:                              29.53
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    1.19e-11
Time:                        15:51:00   Log-Likelihood:                         -189.23
No. Observations:                  60   AIC:                                      384.5
Df Residuals:                      57   BIC:                                      390.7
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.4798      0.166      2.893      0.005       0.148       0.812
HML            0.2177      0.088      2.481      0.016       0.042       0.393
Rm-Rf          1.3127      0.140      9.405      0.000       1.033       1.592
==============================================================================
Omnibus:                        3.616   Durbin-Watson:                   1.652
Prob(Omnibus):                  0.164   Jarque-Bera (JB):                2.689
Skew:                           0.468   Prob(JB):                        0.261
Kurtosis:                       3.445   Cond. No.                         2.23
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
X = final_data2[['SMB','HML','Rm-Rf']]
y = final_data2['BM'] - rf['Interest Rate'].iloc[68:128]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.771
Model:                            OLS   Adj. R-squared (uncentered):              0.759
Method:                 Least Squares   F-statistic:                              63.85
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    3.19e-18
Time:                        15:51:00   Log-Likelihood:                         -175.66
No. Observations:                  60   AIC:                                      357.3
Df Residuals:                      57   BIC:                                      363.6
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB           -0.0959      0.132     -0.725      0.471      -0.361       0.169
HML            0.0033      0.070      0.047      0.962      -0.137       0.143
Rm-Rf          1.3550      0.111     12.172      0.000       1.132       1.578
==============================================================================
Omnibus:                       53.679   Durbin-Watson:                   1.872
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              422.201
Skew:                           2.223   Prob(JB):                     2.09e-92
Kurtosis:                      15.211   Cond. No.                         2.23
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
X = final_data2[['SMB','HML','Rm-Rf']]
y = final_data2['SM'] - rf['Interest Rate'].iloc[68:128]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.771
Model:                            OLS   Adj. R-squared (uncentered):              0.759
Method:                 Least Squares   F-statistic:                              64.13
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    2.90e-18
Time:                        15:51:00   Log-Likelihood:                         -167.35
No. Observations:                  60   AIC:                                      340.7
Df Residuals:                      57   BIC:                                      347.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.4411      0.115      3.829      0.000       0.210       0.672
HML           -0.0139      0.061     -0.228      0.820      -0.136       0.108
Rm-Rf          1.2788      0.097     13.193      0.000       1.085       1.473
==============================================================================
Omnibus:                        2.553   Durbin-Watson:                   1.938
Prob(Omnibus):                  0.279   Jarque-Bera (JB):                1.495
Skew:                          -0.010   Prob(JB):                        0.473
Kurtosis:                       2.227   Cond. No.                         2.23
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
X = final_data2[['SMB','HML','Rm-Rf']]
y = final_data2['HV'] - rf['Interest Rate'].iloc[68:128]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.807
Model:                            OLS   Adj. R-squared (uncentered):              0.796
Method:                 Least Squares   F-statistic:                              79.21
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    2.56e-20
Time:                        15:51:00   Log-Likelihood:                         -198.05
No. Observations:                  60   AIC:                                      402.1
Df Residuals:                      57   BIC:                                      408.4
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB           -0.7517      0.192     -3.912      0.000      -1.136      -0.367
HML           -0.7909      0.102     -7.781      0.000      -0.994      -0.587
Rm-Rf          1.2746      0.162      7.883      0.000       0.951       1.598
==============================================================================
Omnibus:                        2.153   Durbin-Watson:                   1.831
Prob(Omnibus):                  0.341   Jarque-Bera (JB):                1.347
Skew:                           0.287   Prob(JB):                        0.510
Kurtosis:                       3.457   Cond. No.                         2.23
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
X = final_data2[['SMB','HML','Rm-Rf']]
y = final_data2['SV'] - rf['Interest Rate'].iloc[68:128]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.820
Model:                            OLS   Adj. R-squared (uncentered):              0.810
Method:                 Least Squares   F-statistic:                              86.49
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    3.37e-21
Time:                        15:51:00   Log-Likelihood:                         -162.28
No. Observations:                  60   AIC:                                      330.6
Df Residuals:                      57   BIC:                                      336.9
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.4108      0.106      3.881      0.000       0.199       0.623
HML           -0.2665      0.056     -4.760      0.000      -0.379      -0.154
Rm-Rf          1.1873      0.089     13.329      0.000       1.009       1.366
==============================================================================
Omnibus:                        1.302   Durbin-Watson:                   2.147
Prob(Omnibus):                  0.522   Jarque-Bera (JB):                1.087
Skew:                          -0.105   Prob(JB):                        0.581
Kurtosis:                       2.375   Cond. No.                         2.23
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [37]:
# 2013.9 ~ 2018.8

In [38]:
final_data3 = final_data.iloc[128:188]
final_data3

,SMB,HML,Rm-Rf,SH,SM,SV,BH,BM,HV
Symbol,,,,,,,,,
2013-09-30,-0.336371,1.973219,0.807259,2.442637,1.057663,1.615950,2.757632,3.729850,-0.362119
2013-10-31,2.143562,4.600909,-0.997578,2.185331,-2.006150,-2.504204,-1.901225,-0.440974,-6.413508
2013-11-30,-4.369590,0.812138,-2.808694,-3.145930,-2.757832,-4.959163,1.996437,-1.935986,2.185395
2013-12-31,-0.730257,2.038304,-3.990115,-3.158422,-3.306346,-2.540215,-0.238879,-1.641638,-4.933695
2014-01-31,3.255656,4.900903,-5.253435,5.915022,3.497298,1.869598,4.337193,-1.403055,-1.419188
2014-02-28,3.634629,3.465492,0.465452,5.587925,4.588336,4.864258,3.975467,2.393014,-2.231850
2014-03-31,4.280348,4.658007,-2.248057,5.054601,4.437520,2.653592,2.111018,1.997639,-4.803986
2014-04-30,3.266828,4.320710,-3.369799,3.703011,2.110262,1.912442,2.540723,-0.305364,-4.310129
2014-05-31,-0.302720,4.755076,-0.978442,2.152766,-1.468812,-1.497020,2.819080,0.317300,-3.041286


In [39]:
X = final_data3[['SMB','HML','Rm-Rf']]
y = final_data3['BH'] - rf['Interest Rate'].iloc[128:188]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.817
Model:                            OLS   Adj. R-squared (uncentered):              0.808
Method:                 Least Squares   F-statistic:                              85.08
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    4.94e-21
Time:                        15:51:00   Log-Likelihood:                         -135.27
No. Observations:                  60   AIC:                                      276.5
Df Residuals:                      57   BIC:                                      282.8
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB           -0.2609      0.104     -2.508      0.015      -0.469      -0.053
HML            1.0015      0.065     15.395      0.000       0.871       1.132
Rm-Rf          1.1706      0.123      9.519      0.000       0.924       1.417
==============================================================================
Omnibus:                        3.148   Durbin-Watson:                   1.707
Prob(Omnibus):                  0.207   Jarque-Bera (JB):                2.743
Skew:                           0.133   Prob(JB):                        0.254
Kurtosis:                       4.013   Cond. No.                         2.20
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
X = final_data3[['SMB','HML','Rm-Rf']]
y = final_data3['SH'] - rf['Interest Rate'].iloc[128:188]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.847
Model:                            OLS   Adj. R-squared (uncentered):              0.839
Method:                 Least Squares   F-statistic:                              105.4
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    3.12e-23
Time:                        15:51:00   Log-Likelihood:                         -127.85
No. Observations:                  60   AIC:                                      261.7
Df Residuals:                      57   BIC:                                      268.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            1.0028      0.092     10.907      0.000       0.819       1.187
HML            0.7538      0.057     13.112      0.000       0.639       0.869
Rm-Rf          1.0633      0.109      9.785      0.000       0.846       1.281
==============================================================================
Omnibus:                        0.438   Durbin-Watson:                   1.913
Prob(Omnibus):                  0.803   Jarque-Bera (JB):                0.133
Skew:                          -0.106   Prob(JB):                        0.936
Kurtosis:                       3.092   Cond. No.                         2.20
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [41]:
X = final_data3[['SMB','HML','Rm-Rf']]
y = final_data3['BM'] - rf['Interest Rate'].iloc[128:188]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.771
Model:                            OLS   Adj. R-squared (uncentered):              0.759
Method:                 Least Squares   F-statistic:                              63.87
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    3.17e-18
Time:                        15:51:00   Log-Likelihood:                         -105.33
No. Observations:                  60   AIC:                                      216.7
Df Residuals:                      57   BIC:                                      222.9
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.1007      0.063      1.595      0.116      -0.026       0.227
HML            0.1572      0.039      3.980      0.000       0.078       0.236
Rm-Rf          1.0188      0.075     13.645      0.000       0.869       1.168
==============================================================================
Omnibus:                        2.487   Durbin-Watson:                   2.106
Prob(Omnibus):                  0.288   Jarque-Bera (JB):                2.116
Skew:                          -0.338   Prob(JB):                        0.347
Kurtosis:                       2.377   Cond. No.                         2.20
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [42]:
X = final_data3[['SMB','HML','Rm-Rf']]
y = final_data3['SM'] - rf['Interest Rate'].iloc[128:188]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.828
Model:                            OLS   Adj. R-squared (uncentered):              0.819
Method:                 Least Squares   F-statistic:                              91.63
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    8.72e-22
Time:                        15:51:00   Log-Likelihood:                         -120.29
No. Observations:                  60   AIC:                                      246.6
Df Residuals:                      57   BIC:                                      252.9
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.9604      0.081     11.850      0.000       0.798       1.123
HML            0.2142      0.051      4.227      0.000       0.113       0.316
Rm-Rf          1.1133      0.096     11.621      0.000       0.921       1.305
==============================================================================
Omnibus:                        7.234   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.027   Jarque-Bera (JB):                6.736
Skew:                          -0.613   Prob(JB):                       0.0345
Kurtosis:                       4.091   Cond. No.                         2.20
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
X = final_data3[['SMB','HML','Rm-Rf']]
y = final_data3['HV'] - rf['Interest Rate'].iloc[128:188]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.708
Model:                            OLS   Adj. R-squared (uncentered):              0.692
Method:                 Least Squares   F-statistic:                              46.01
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    3.08e-15
Time:                        15:51:00   Log-Likelihood:                         -136.04
No. Observations:                  60   AIC:                                      278.1
Df Residuals:                      57   BIC:                                      284.4
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB           -0.0674      0.105     -0.639      0.525      -0.278       0.144
HML           -0.2177      0.066     -3.304      0.002      -0.350      -0.086
Rm-Rf          1.1106      0.125      8.915      0.000       0.861       1.360
==============================================================================
Omnibus:                        1.431   Durbin-Watson:                   1.923
Prob(Omnibus):                  0.489   Jarque-Bera (JB):                0.826
Skew:                          -0.258   Prob(JB):                        0.662
Kurtosis:                       3.252   Cond. No.                         2.20
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [44]:
X = final_data3[['SMB','HML','Rm-Rf']]
y = final_data3['SV'] - rf['Interest Rate'].iloc[128:188]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.814
Model:                            OLS   Adj. R-squared (uncentered):              0.804
Method:                 Least Squares   F-statistic:                              83.04
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    8.66e-21
Time:                        15:51:01   Log-Likelihood:                         -122.63
No. Observations:                  60   AIC:                                      251.3
Df Residuals:                      57   BIC:                                      257.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.8092      0.084      9.603      0.000       0.640       0.978
HML           -0.0270      0.053     -0.512      0.610      -0.133       0.079
Rm-Rf          1.1233      0.100     11.277      0.000       0.924       1.323
==============================================================================
Omnibus:                        3.852   Durbin-Watson:                   1.685
Prob(Omnibus):                  0.146   Jarque-Bera (JB):                3.130
Skew:                           0.334   Prob(JB):                        0.209
Kurtosis:                       3.897   Cond. No.                         2.20
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [45]:
# 2018.9 ~ 2023.8

In [46]:
final_data4 = final_data.iloc[188:248]
final_data4

,SMB,HML,Rm-Rf,SH,SM,SV,BH,BM,HV
Symbol,,,,,,,,,
2018-09-30,-2.452162,3.234343,-0.026822,1.188087,0.591703,0.119243,6.582858,1.489644,1.183016
2018-10-31,-6.862198,-3.738728,-13.034816,-21.312957,-21.514833,-21.438713,-20.183986,-10.915150,-12.580774
2018-11-30,5.945772,4.725843,1.749012,13.479537,13.573734,8.999886,9.570534,4.046808,4.598499
2018-12-31,0.759334,-0.828715,-4.476907,-1.971462,-1.084121,-0.480213,-0.037210,-5.905560,0.128973
2019-01-31,2.703339,0.294891,5.133806,11.929615,10.566626,9.346673,5.305144,11.129449,7.298304
2019-02-28,-1.876854,1.885447,-1.041813,-0.036416,-0.097066,-0.850253,3.266938,1.070010,0.309880
2019-03-31,1.566118,4.763090,-3.867573,2.115182,-0.369567,-2.716568,0.092215,-1.159306,-4.602215
2019-04-30,1.186651,1.699472,0.747733,7.705090,3.846011,4.104572,4.093412,3.707320,4.294987
2019-05-31,1.038668,1.938191,-8.000288,-3.754963,-6.142180,-7.499429,-6.318341,-7.743978,-6.450256


In [47]:
X = final_data4[['SMB','HML','Rm-Rf']]
y = final_data4['BH'] - rf['Interest Rate'].iloc[188:248]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.929
Model:                            OLS   Adj. R-squared (uncentered):              0.925
Method:                 Least Squares   F-statistic:                              247.9
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    1.16e-32
Time:                        15:51:01   Log-Likelihood:                         -133.25
No. Observations:                  60   AIC:                                      272.5
Df Residuals:                      57   BIC:                                      278.8
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB           -0.0018      0.078     -0.023      0.981      -0.158       0.154
HML            1.0839      0.066     16.421      0.000       0.952       1.216
Rm-Rf          1.2035      0.056     21.455      0.000       1.091       1.316
==============================================================================
Omnibus:                        0.907   Durbin-Watson:                   1.786
Prob(Omnibus):                  0.635   Jarque-Bera (JB):                0.960
Skew:                           0.270   Prob(JB):                        0.619
Kurtosis:                       2.697   Cond. No.                         1.81
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
X = final_data4[['SMB','HML','Rm-Rf']]
y = final_data4['SH'] - rf['Interest Rate'].iloc[188:248]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.970
Model:                            OLS   Adj. R-squared (uncentered):              0.968
Method:                 Least Squares   F-statistic:                              609.5
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    2.96e-43
Time:                        15:51:01   Log-Likelihood:                         -105.54
No. Observations:                  60   AIC:                                      217.1
Df Residuals:                      57   BIC:                                      223.4
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            1.1875      0.049     24.183      0.000       1.089       1.286
HML            0.7075      0.042     17.007      0.000       0.624       0.791
Rm-Rf          1.0373      0.035     29.344      0.000       0.966       1.108
==============================================================================
Omnibus:                        1.730   Durbin-Watson:                   1.231
Prob(Omnibus):                  0.421   Jarque-Bera (JB):                1.221
Skew:                          -0.046   Prob(JB):                        0.543
Kurtosis:                       2.307   Cond. No.                         1.81
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:
X = final_data4[['SMB','HML','Rm-Rf']]
y = final_data4['BM'] - rf['Interest Rate'].iloc[188:248]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.913
Model:                            OLS   Adj. R-squared (uncentered):              0.909
Method:                 Least Squares   F-statistic:                              199.8
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    3.32e-30
Time:                        15:51:01   Log-Likelihood:                         -123.58
No. Observations:                  60   AIC:                                      253.2
Df Residuals:                      57   BIC:                                      259.4
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB           -0.0011      0.066     -0.017      0.987      -0.134       0.132
HML            0.0823      0.056      1.465      0.149      -0.030       0.195
Rm-Rf          1.1431      0.048     23.943      0.000       1.048       1.239
==============================================================================
Omnibus:                        5.156   Durbin-Watson:                   1.742
Prob(Omnibus):                  0.076   Jarque-Bera (JB):                4.656
Skew:                           0.421   Prob(JB):                       0.0975
Kurtosis:                       4.074   Cond. No.                         1.81
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [50]:
X = final_data4[['SMB','HML','Rm-Rf']]
y = final_data4['SM'] - rf['Interest Rate'].iloc[188:248]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.976
Model:                            OLS   Adj. R-squared (uncentered):              0.975
Method:                 Least Squares   F-statistic:                              773.9
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    3.96e-46
Time:                        15:51:01   Log-Likelihood:                         -103.28
No. Observations:                  60   AIC:                                      212.6
Df Residuals:                      57   BIC:                                      218.9
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            1.1269      0.047     23.831      0.000       1.032       1.222
HML            0.2664      0.040      6.650      0.000       0.186       0.347
Rm-Rf          1.1980      0.034     35.190      0.000       1.130       1.266
==============================================================================
Omnibus:                        5.925   Durbin-Watson:                   1.424
Prob(Omnibus):                  0.052   Jarque-Bera (JB):                5.095
Skew:                          -0.555   Prob(JB):                       0.0783
Kurtosis:                       3.898   Cond. No.                         1.81
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [51]:
X = final_data4[['SMB','HML','Rm-Rf']]
y = final_data4['HV'] - rf['Interest Rate'].iloc[188:248]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.927
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              241.2
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    2.38e-32
Time:                        15:51:01   Log-Likelihood:                         -119.86
No. Observations:                  60   AIC:                                      245.7
Df Residuals:                      57   BIC:                                      252.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.2515      0.062      4.034      0.000       0.127       0.376
HML           -0.2005      0.053     -3.796      0.000      -0.306      -0.095
Rm-Rf          1.0647      0.045     23.724      0.000       0.975       1.155
==============================================================================
Omnibus:                        1.658   Durbin-Watson:                   1.419
Prob(Omnibus):                  0.436   Jarque-Bera (JB):                0.937
Skew:                           0.089   Prob(JB):                        0.626
Kurtosis:                       3.586   Cond. No.                         1.81
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [52]:
X = final_data4[['SMB','HML','Rm-Rf']]
y = final_data4['SV'] - rf['Interest Rate'].iloc[188:248]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.972
Model:                            OLS   Adj. R-squared (uncentered):              0.971
Method:                 Least Squares   F-statistic:                              665.7
Date:                Thu, 12 Oct 2023   Prob (F-statistic):                    2.59e-44
Time:                        15:51:01   Log-Likelihood:                         -105.45
No. Observations:                  60   AIC:                                      216.9
Df Residuals:                      57   BIC:                                      223.2
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SMB            0.9342      0.049     19.055      0.000       0.836       1.032
HML           -0.0081      0.042     -0.196      0.845      -0.091       0.075
Rm-Rf          1.1761      0.035     33.323      0.000       1.105       1.247
==============================================================================
Omnibus:                        3.068   Durbin-Watson:                   1.646
Prob(Omnibus):                  0.216   Jarque-Bera (JB):                2.515
Skew:                           0.500   Prob(JB):                        0.284
Kurtosis:                       3.083   Cond. No.                         1.81
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""